In [2]:
%load_ext autoreload

In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import protfasta 
%autoreload 2
%aimport AD_predictor_tools
%aimport AD_comparison_tools
%aimport PlottingTools

1. Adding uniprot isoform IDs to lambert TFs

In [4]:
# Loading in Lambert TFs
lambert_TFs = pd.read_csv("../data/LambertTFs.csv", index_col = 0)
lambert_TFs

,GeneName,ProteinSeq
0,sp|P23511|NFYA_HUMAN,MEQYTANSNSSTEQIVVQAGQIQQQQQGGVTAVQLQTEAQVASASG...
1,sp|Q96QS3|ARX_HUMAN,MSNQYQEEGCSERPECKSKSPTLLSSYCIDSILGRRSPCKMRLLGA...
2,sp|P31270|HXA11_HUMAN,MDFDERGPCSSNMYLPSCTYYVSGPDFSSLPSFLPQTPSSRPMTYS...
3,sp|P50221|MEOX1_HUMAN,MDPAASSCMRSLQPPAPVWGCLRNPHSEGNGASGLPHYPPTPFSFH...
4,sp|P57073|SOX8_HUMAN,MLDMSEARSQPPCSPSGTASSMSHVEDSDSDAPPSPAGSEGLGRAG...
...,...,...
1603,sp|P35227|PCGF2_HUMAN,MHRTTRIKITELNPHLMCALCGGYFIDATTIVECLHSFCKTCIVRY...
1604,sp|Q9BS34|ZN670_HUMAN,MDSVSFEDVAVAFTQEEWALLDPSQKNLYRDVMQEIFRNLASVGNK...
1605,sp|P17098|ZNF8_HUMAN,MDPEDEGVAGVMSVGPPAARLQEPVTFRDVAVDFTQEEWGQLDPTQ...
1606,sp|Q9UJW7|ZN229_HUMAN,METLTSRHEKRALHSQASAISQDREEKIMSQEPLSFKDVAVVFTEE...


Found uniprotIDs of TFs in this notebook: http://localhost:8888/notebooks/Desktop/Staller_Lab/SFARI/notebooks/Replicating%20Soto%20gnomAD%20Analysis%20-%20ENST%20Codes.ipynb

But will do again here to check

In [12]:
# Downloading uniprotIDs
lambert_TFs["uniprotID"] = lambert_TFs["GeneName"].str.split("|").str[1]
lambert_TFs[["uniprotID"]].to_csv("../data/lambert_TF_uniprotIDs.txt", 
                                                 header=None, index=None, sep=' ', mode='a')
lambert_TFs

,GeneName,Sequence,uniprotID,Start,End,matching_isoforms
0,sp|P23511|NFYA_HUMAN,MEQYTANSNSSTEQIVVQAGQIQQQQQGGVTAVQLQTEAQVASASG...,P23511,1,347,
1,sp|Q96QS3|ARX_HUMAN,MSNQYQEEGCSERPECKSKSPTLLSSYCIDSILGRRSPCKMRLLGA...,Q96QS3,1,562,
2,sp|P31270|HXA11_HUMAN,MDFDERGPCSSNMYLPSCTYYVSGPDFSSLPSFLPQTPSSRPMTYS...,P31270,1,313,
3,sp|P50221|MEOX1_HUMAN,MDPAASSCMRSLQPPAPVWGCLRNPHSEGNGASGLPHYPPTPFSFH...,P50221,1,254,
4,sp|P57073|SOX8_HUMAN,MLDMSEARSQPPCSPSGTASSMSHVEDSDSDAPPSPAGSEGLGRAG...,P57073,1,446,
...,...,...,...,...,...,...
1603,sp|P35227|PCGF2_HUMAN,MHRTTRIKITELNPHLMCALCGGYFIDATTIVECLHSFCKTCIVRY...,P35227,1,344,
1604,sp|Q9BS34|ZN670_HUMAN,MDSVSFEDVAVAFTQEEWALLDPSQKNLYRDVMQEIFRNLASVGNK...,Q9BS34,1,389,
1605,sp|P17098|ZNF8_HUMAN,MDPEDEGVAGVMSVGPPAARLQEPVTFRDVAVDFTQEEWGQLDPTQ...,P17098,1,575,
1606,sp|Q9UJW7|ZN229_HUMAN,METLTSRHEKRALHSQASAISQDREEKIMSQEPLSFKDVAVVFTEE...,Q9UJW7,1,825,


In [14]:
# Adding start and end
lambert_TFs["Start"] = 1
lambert_TFs["End"] = lambert_TFs["Sequence"].str.len()
#lambert_TFs = lambert_TFs.rename(columns = {"ProteinSeq" : "Sequence"})
lambert_TFs["uniprotID"] = lambert_TFs["GeneName"].str.split("|").str[1]
lambert_TFs

,GeneName,Sequence,uniprotID,Start,End,matching_isoforms
0,sp|P23511|NFYA_HUMAN,MEQYTANSNSSTEQIVVQAGQIQQQQQGGVTAVQLQTEAQVASASG...,P23511,1,347,
1,sp|Q96QS3|ARX_HUMAN,MSNQYQEEGCSERPECKSKSPTLLSSYCIDSILGRRSPCKMRLLGA...,Q96QS3,1,562,
2,sp|P31270|HXA11_HUMAN,MDFDERGPCSSNMYLPSCTYYVSGPDFSSLPSFLPQTPSSRPMTYS...,P31270,1,313,
3,sp|P50221|MEOX1_HUMAN,MDPAASSCMRSLQPPAPVWGCLRNPHSEGNGASGLPHYPPTPFSFH...,P50221,1,254,
4,sp|P57073|SOX8_HUMAN,MLDMSEARSQPPCSPSGTASSMSHVEDSDSDAPPSPAGSEGLGRAG...,P57073,1,446,
...,...,...,...,...,...,...
1603,sp|P35227|PCGF2_HUMAN,MHRTTRIKITELNPHLMCALCGGYFIDATTIVECLHSFCKTCIVRY...,P35227,1,344,
1604,sp|Q9BS34|ZN670_HUMAN,MDSVSFEDVAVAFTQEEWALLDPSQKNLYRDVMQEIFRNLASVGNK...,Q9BS34,1,389,
1605,sp|P17098|ZNF8_HUMAN,MDPEDEGVAGVMSVGPPAARLQEPVTFRDVAVDFTQEEWGQLDPTQ...,P17098,1,575,
1606,sp|Q9UJW7|ZN229_HUMAN,METLTSRHEKRALHSQASAISQDREEKIMSQEPLSFKDVAVVFTEE...,Q9UJW7,1,825,


In [17]:
# Importing the sequences of all isoforms of the uniprotIDs from uniprot
TF_isoform_seqs = protfasta.read_fasta("../data/all_lambert_TF_isoforms.txt")
TF_isoform_seqs_df = pd.DataFrame({"id" : TF_isoform_seqs.keys(),
                                  "seq" : TF_isoform_seqs.values()})
TF_isoform_seqs_df["uniprotID"] = TF_isoform_seqs_df["id"].str.split("|").str[1]

TF_isoform_seqs_df

,id,seq,uniprotID
0,sp|A0AVK6|E2F8_HUMAN Transcription factor E2F8...,MENEKENLFCEPHKRGLMKTPLKESTTANIVLAEIQPDFGPLTTPT...,A0AVK6
1,sp|A0PJY2|FEZF1_HUMAN Fez family zinc finger p...,MDSSCHNATTKMLATAPARGNMMSTSKPLAFSIERIMARTPEPKAL...,A0PJY2
2,sp|A0PJY2-2|FEZF1_HUMAN Isoform 2 of Fez famil...,MDSSCHNATTKMLATAPARGNMMSTSKPLAFSIERIMARTPEPKAL...,A0PJY2-2
3,sp|A0PJY2-3|FEZF1_HUMAN Isoform 3 of Fez famil...,MDSSCHNATTKMLATAPARGNMMSTSKPLAFSIERIMARTPEPKAL...,A0PJY2-3
4,sp|A1A519|F170A_HUMAN Protein FAM170A OS=Homo ...,MKRRQKRKHLENEESQETAEKGGGMSKSQEDALQPGSTRVAKGWSQ...,A1A519
...,...,...,...
3526,sp|Q9Y6X8|ZHX2_HUMAN Zinc fingers and homeobox...,MASKRKSTTPCMVRTSQVVEQDVPEEVDRAKEKGIGTPQPDVAKDS...,Q9Y6X8
3527,sp|Q9Y6Y1|CMTA1_HUMAN Calmodulin-binding trans...,MWRAEGKWLPKTSRKSVSQSVFCGTSTYCVLNTVPPIEDDHGNSNS...,Q9Y6Y1
3528,sp|Q9Y6Y1-2|CMTA1_HUMAN Isoform 2 of Calmoduli...,MWRAEGKWLPKTSRKSVSQSVFCGTSTYCVLNTVPPIEDDHGNSNS...,Q9Y6Y1-2
3529,sp|Q9Y6Y1-3|CMTA1_HUMAN Isoform 3 of Calmoduli...,MWRAEGKWLPKTSRKSVSQSVFCGTSTYCVLNTVPPIEDDHGNSNS...,Q9Y6Y1-3


In [18]:
mapped_lambert_TFs = AD_comparison_tools.return_uniprotID_isoform_mappings(lambert_TFs, TF_isoform_seqs_df)
mapped_lambert_TFs

no matching isoforms for:
Q92782
Tested:
2561      Q92782
2562    Q92782-1
2563    Q92782-3
Name: uniprotID, dtype: object
1
380
expected:
Q92782
MGGLSARPTAGRTDPAGTCWGQDPGSKMATVIPGPLSLGEDFYREAIEHCRSYNARLCAERSLRLPFLDSQTGVAQNNCYIWMEKTHRGPGLAPGQIYTYPARCWRKKRRLNILEDPRLRPCEYKIDCEAPLKKEGGLPEGPVLEALLCAETGEKKIELKEEETIMDCQKQQLLEFPHDLEVEDLEDDIPRRKNRAKGKAYGIGGLRKRQDTASLEDRDKPYVCDKFYKELAWVPEAQRKHTAKKAPDGTVIPNGYCDFCLGGSKKTGCPEDLISCADCGRSGHPSCLQFTVNMTAAVRTYRWQCIECKSCSLCGTSENDGASWAGLTPQDQLLFCDDCDRGYHMYCLSPPMAEPPEGSWSCHLCLRHLKEKASAYITLT
observed:
['MATVIPGPLSLGEDFYREAIEHCRSYNARLCAERSLRLPFLDSQTGVAQNNCYIWMEKTHRGPGLAPGQIYTYPARCWRKKRRLNILEDPRLRPCEYKIDCEAPLKKEGGLPEGPVLEALLCAETGEKKIELKEEETIMDCQKQQLLEFPHDLEVEDLEDDIPRRKNRAKGKAYGIGGLRKRQDTASLEDRDKPYVCDICGKRYKNRPGLSYHYTHTHLAEEEGEENAERHALPFHRKNNHKQFYKELAWVPEAQRKHTAKKAPDGTVIPNGYCDFCLGGSKKTGCPEDLISCADCGRSGHPSCLQFTVNMTAAVRTYRWQCIECKSCSLCGTSENDDQLLFCDDCDRGYHMYCLSPPMAEPPEGSWSCHLCLRHLKEKA', '', '']

no matching isoforms for:
P15822
Tested:
622      P15822
623    P1582

no matching isoforms for:
Q96BR9
Tested:
2602      Q96BR9
2603    Q96BR9-2
Name: uniprotID, dtype: object
1
441
expected:
Q96BR9
MEISSHQSHLLQQLNEQRRQDVFCDCSILVEGKVFKAHRNVLFASSGYFKMLLSQNSKETSQPTTATFQAFSPDTFTVILDFVYSGKLSLTGQNVIEVMSAASFLQMTDVISVCKTFIKSSLDISEKEKDRYFSLSDKDANSNGVERSSFYSGGWQEGSSSPRSHLSPEQGTGIISGKSWNKYNYHPASQKNTQQPLAKHEPRKESIKKTKHLRLSQPSEVTHYKSSKREVRTSDSSSHVSQSEEQAQIDAEMDSTPVGYQYGQGSDVTSKSFPDDLPRMRFKCPYCTHVVKRKADLKRHLRCHTGERPYPCQACGKRFSRLDHLSSHFRTIHQACKLICRKCKRHVTDLTGQVVQEGTRRYRLCNECLAEFGIDSLPIDLEAEQHLMSPSDGDKDSRWHLSEDENRSYVEIVEDGSGDLVIQQVDDSEEEEEKEIKPNIR
observed:
['MEISSHQSHLLQQLNEQRRQDVFCDCSILVEGKVFKAHRNVLFASSGYFKMLLSQNSKETSQPTTATFQAFSPDTFTVILDFVYSGKLSLTGQNVIEVMSAASFLQMTDVISVCKTFIKSSLDISEKEKDRYFSLSDKDANSNGVERSSFYSGGWQEGSSSPRSHLSPEQGTGIISGKSWNKYNYHPASQKNTQQPLAKHEPRKESIKKTKHLRLSQPSEVTHYKSSKREVRTSDSSSHVSQSEEQAQIDAEMDSTPVGYQYGQGSDVTSKSFPDDLPRMRFKCPYCTHVVKRKADLKRHLRCHTGERPYPCQACGKRFSRLDHLSSHFRTIHQACKLICRKCKRHVTDLTGQVVQEGTRRYRLCNECLAEFGIDSLPIDLEAEQHLMSPSDGDKDSRWHLSEDENRSYVEIVEDGS

no matching isoforms for:
P17019
Tested:
669    P17019
Name: uniprotID, dtype: object
1
499
expected:
P17019
MKRHEMAAKPPAMCSHFAKDLRPEQYIKNSFQQVILRRYGKCGYQKGCKSVDEHKLHKGGHKGLNRCVTTTQSKIVQCDKYVKVFHKYSNAKRHKIRHTGKNPFKCKECGKSFCMLSQLTQHEIIHTGEKPYKCEECGKAFKKSSNLTNHKIIHTGEKPYKCEECGKAFNQSSTLTRHKIIHTGEKLYKCEECGKAFNRSSNLTKHKIVHTGEKPYKCEECGKAFKQSSNLTNHKKIHTGEKPYKCGECGKAFTLSSHLTTHKRIHTGEKPYKCEECGKAFSVFSTLTKHKIIHTEEKPYKCEECGKAFNRSSHLTNHKVIHTGEKPYKCEECGKAFTKSSTLTYHKVIHTGKKPYKCEECGKAFSIFSILTKHKVIHTEDKPYKCEECGKTFNYSSNFTNHKKIHTGEKPYKCEECGKSFILSSHLTTHKIIHTGEKPYKCKECGKAFNQSSTLMKHKIIHTGEKPYKCEECGKAFNQSPNLTKHKRIHTKEKPYKCK
observed:
['MGPLTFMDVAIEFSLEEWQCLDTAQQNLYRNVMLENYRNLVFLGIAVSNLDLITCLEQGKEPWNMKRHEMAAKPPAMCSHFAKDLRPEQYIKNSFQQVILRRYGKCGYQKGCKSVDEHKLHKGGHKGLNRCVTTTQSKIVQCDKYVKVFHKYSNAKRHKIRHTGKNPFKCKECGKSFCMLSQLTQHEIIHTGEKPYKCEECGKAFKKSSNLTNHKIIHTGEKPYKCEECGKAFNQSSTLTRHKIIHTGEKLYKCEECGKAFNRSSNLTKHKIVHTGEKPYKCEECGKAFKQSSNLTNHKKIHTGEKPYKCGECGKAFTLSSHLTTHKRIHTGEKPYKCEECGKAFSVFSTLTKHKIIHTEEKPYKCEECGKAFNRSSHL

no matching isoforms for:
A6NNF4
Tested:
34      A6NNF4
35    A6NNF4-2
Name: uniprotID, dtype: object
1
738
expected:
A6NNF4
MGLLTFRDVAIEFSLEEWQCLDTAQKNLYRNVMLENYRNLAFLGIAVSKPDLIICLEKEKEPWNMKRDEMVDEPPGICPHFAQDIWPEQGVEDSFQKVILRRFEKCGHENLQLRKGCKSVDECKVHKEGYNGLNQCFTTTQGKASQCGKYLKVFYKFINLNRYKIRHTRKKPFKCKNCVKSFCMFSHKTQHKSIYTTEKSYKCKECGKTFNWSSTLTNHKKTHTEEKPYKCEEYGKAFNQSSNYTTHKVTHTGEKPYKCEECGKAFSQSSTLTIHKRIHTGEKPCKCEECGKAFSQPSALTIHKRMHIGEKPYKCEECGKAFVWSSTLTRHKRLHSGEKPYKCEECAKAFSQFGHLTTHRIIHTGEKPYKCEECGKAFIWPSTLTKHKRIHTGEKPYKCEECGKAFHRSSNLTKHKIIHTGEKPYKCEECGKAFIWSSNLTEHKKIHTREKPYKCEECSKAFSRSSALTTHKRMHTGEKPYKCEECGKAFSQSSTLTAHKIIHTGEKPYKCEECGKAFILSSTLSKHKRIHTGEKPYKCEECGKTFNQSSNLSTHKIIHTGEKPYKCEECGKAFNRSSNLSTHKIIHTGEKPYKCDECGKSFIWSSTLFKHKRIHTGEKPYKCEECGKAFNHSQILLHIRHKRMHTGEKPYKCEECGKSFNLSSTFIKHKVIHTGVKLYKCEECGKVFFWSSALTRHKKIHAGQQPYKWEKIGKAFNQSSHLTTDKITHIGEKSYKCE
observed:
['', '']

no matching isoforms for:
Q2VWA4
Tested:
1789      Q2VWA4
1790    Q2VWA4-2
Name: uniprotID, dtype: object
1
1001
exp

,GeneName,Sequence,uniprotID,Start,End,matching_isoforms
0,sp|P23511|NFYA_HUMAN,MEQYTANSNSSTEQIVVQAGQIQQQQQGGVTAVQLQTEAQVASASG...,P23511,1,347,[P23511]
1,sp|Q96QS3|ARX_HUMAN,MSNQYQEEGCSERPECKSKSPTLLSSYCIDSILGRRSPCKMRLLGA...,Q96QS3,1,562,[Q96QS3]
2,sp|P31270|HXA11_HUMAN,MDFDERGPCSSNMYLPSCTYYVSGPDFSSLPSFLPQTPSSRPMTYS...,P31270,1,313,[P31270]
3,sp|P50221|MEOX1_HUMAN,MDPAASSCMRSLQPPAPVWGCLRNPHSEGNGASGLPHYPPTPFSFH...,P50221,1,254,[P50221]
4,sp|P57073|SOX8_HUMAN,MLDMSEARSQPPCSPSGTASSMSHVEDSDSDAPPSPAGSEGLGRAG...,P57073,1,446,[P57073]
...,...,...,...,...,...,...
1603,sp|P35227|PCGF2_HUMAN,MHRTTRIKITELNPHLMCALCGGYFIDATTIVECLHSFCKTCIVRY...,P35227,1,344,[P35227]
1604,sp|Q9BS34|ZN670_HUMAN,MDSVSFEDVAVAFTQEEWALLDPSQKNLYRDVMQEIFRNLASVGNK...,Q9BS34,1,389,[Q9BS34]
1605,sp|P17098|ZNF8_HUMAN,MDPEDEGVAGVMSVGPPAARLQEPVTFRDVAVDFTQEEWGQLDPTQ...,P17098,1,575,[P17098]
1606,sp|Q9UJW7|ZN229_HUMAN,METLTSRHEKRALHSQASAISQDREEKIMSQEPLSFKDVAVVFTEE...,Q9UJW7,1,825,[]


In [21]:
mapped_lambert_TFs["matching_isoform_count"] = [len(a) for a in mapped_lambert_TFs["matching_isoforms"]]
mapped_lambert_TFs[mapped_lambert_TFs["matching_isoform_count"] == 0]

,GeneName,Sequence,uniprotID,Start,End,matching_isoforms,matching_isoform_count
26,sp|Q92782|DPF1_HUMAN,MGGLSARPTAGRTDPAGTCWGQDPGSKMATVIPGPLSLGEDFYREA...,Q92782,1,380,[],0
144,sp|P15822|ZEP1_HUMAN,MPRTKQIHPRNLRDKIEEAQKELNGAEVSKKEILQAGVKGTSESLK...,P15822,1,2718,[],0
532,sp|Q04741|EMX1_HUMAN,MFQPAAKRGFTIESLVAKDGGTGGGTGGGGAGSHLLAAAASEEPLR...,Q04741,1,257,[],0
595,sp|Q9HBT7|ZN287_HUMAN,MNSSSRSQILLRWKSDKAQSGPYNVEKEILTSRFLRDTETCRQNFR...,Q9HBT7,1,754,[],0
744,sp|Q96BR9|ZBT8A_HUMAN,MEISSHQSHLLQQLNEQRRQDVFCDCSILVEGKVFKAHRNVLFASS...,Q96BR9,1,441,[],0
863,sp|Q96K58|ZN668_HUMAN,MEVEAAEARSPAPGYKRSGRRYKCVSCTKTFPNAPRAARHAATHGP...,Q96K58,1,619,[],0
908,sp|P59817|Z280A_HUMAN,MGDIFLCKKVESPKKNLRESKQREEDDEDPDLIYVGVEHVHRDAEV...,P59817,1,542,[],0
1025,sp|Q14995|NR1D2_HUMAN,MEVNAGGVIAYISSSSSASSPASCHSEGSENSFQSSSSSVPSSPNS...,Q14995,1,579,[],0
1028,sp|Q8IZM8|ZN654_HUMAN,MLCNKEFLGGHIVRHAQAHQKKGSFACVICGRKFRNRGLMQKHLKN...,Q8IZM8,1,581,[],0
1048,sp|Q4G112|HSF5_HUMAN,MEALLSTPINPNNFPAKLWRLVNSPRYRSIRWDGRGEGLLIDQPLF...,Q4G112,1,596,[],0


In [102]:
# Adding uniprotID
TF_isoform_seqs_df["uniprotID"] = TF_isoform_seqs_df["id"].str.split("|").str[1]
TF_isoform_seqs_df

,id,seq,uniprotID
0,sp|A0AVK6|E2F8_HUMAN Transcription factor E2F8...,MENEKENLFCEPHKRGLMKTPLKESTTANIVLAEIQPDFGPLTTPT...,A0AVK6
1,sp|A0PJY2|FEZF1_HUMAN Fez family zinc finger p...,MDSSCHNATTKMLATAPARGNMMSTSKPLAFSIERIMARTPEPKAL...,A0PJY2
2,sp|A0PJY2-2|FEZF1_HUMAN Isoform 2 of Fez famil...,MDSSCHNATTKMLATAPARGNMMSTSKPLAFSIERIMARTPEPKAL...,A0PJY2-2
3,sp|A0PJY2-3|FEZF1_HUMAN Isoform 3 of Fez famil...,MDSSCHNATTKMLATAPARGNMMSTSKPLAFSIERIMARTPEPKAL...,A0PJY2-3
4,sp|A1A519|F170A_HUMAN Protein FAM170A OS=Homo ...,MKRRQKRKHLENEESQETAEKGGGMSKSQEDALQPGSTRVAKGWSQ...,A1A519
...,...,...,...
3526,sp|Q9Y6X8|ZHX2_HUMAN Zinc fingers and homeobox...,MASKRKSTTPCMVRTSQVVEQDVPEEVDRAKEKGIGTPQPDVAKDS...,Q9Y6X8
3527,sp|Q9Y6Y1|CMTA1_HUMAN Calmodulin-binding trans...,MWRAEGKWLPKTSRKSVSQSVFCGTSTYCVLNTVPPIEDDHGNSNS...,Q9Y6Y1
3528,sp|Q9Y6Y1-2|CMTA1_HUMAN Isoform 2 of Calmoduli...,MWRAEGKWLPKTSRKSVSQSVFCGTSTYCVLNTVPPIEDDHGNSNS...,Q9Y6Y1-2
3529,sp|Q9Y6Y1-3|CMTA1_HUMAN Isoform 3 of Calmoduli...,MWRAEGKWLPKTSRKSVSQSVFCGTSTYCVLNTVPPIEDDHGNSNS...,Q9Y6Y1-3


[["Gene ", "AD name", "Start", "End", "uniprotID", "Reference"]]

In [103]:
GSL = pd.read_csv("../output/GSL_with_isoforms.csv")[["GeneName", "Start", "End", "uniprotID", "Reference", "Sequence"]]
GSL["Reference"] = GSL["Reference"] + ", GSL"
GSL

,GeneName,Start,End,uniprotID,Reference,Sequence
0,MEIS1,335,390,O00470,"R4TA_regions.txt / activation_regions.txt, GSL",IDQSNRAVSQGTPYNPDGQPMGGFVMDGQQHMGIRAPGPMSGMGMN...
1,E2F3,391,465,O00716,"activation_regions.txt, GSL",SVSMGNLSPLASPANLLQQTEDQIPSNLEGPFVNLLPPLLQEDYLL...
2,MEIS2,340,477,O14770,"activation_regions.txt, GSL",DQSNRAGFLLDPSVSQGAAYSPEGQPMGSFVLDGQQHMGIRPAGLQ...
3,TFEC,1,119,O14948,"activation_regions.txt, GSL",MTLDHQIINPTLKWSQPAVPSGGPLVQHAHTTLDSDAGLTENPLTK...
4,TFEC,271,347,O14948,"activation_regions.txt, GSL",PSPELCDQAIAFSDPLSYFTDLSFSAALKEEQRLDGMLLDDTISPF...
...,...,...,...,...,...,...
166,DMTF1,87,170,Q9Y222,"R4TA_regions.txt / activation_regions.txt, GSL",VTMTATTEVADDEVTEGTVTQIQILQNEQLDEISPLGNEEVSAVSQ...
167,FOXA2,14,93,Q9Y261,"activation_regions.txt, GSL",DWSSYYAEPEGYSSVSNMNAGLGMNGMNTYMSMSAAAMGSGSGNMS...
168,FOXA2,361,457,Q9Y261,"activation_regions.txt, GSL",EAHLKPEHHYAFNHPFSINNLMSSEQQHHHSHHHHQPHKMDLKAYE...
169,FOXA2/HNF3B,367,463,Q9Y261-2,"Choi 2000 list, GSL",EAHLKPEHHYAFNHPFSINNLMSSEQQHHHSHHHHQPHKMDLKAYE...


In [104]:
Soto = pd.read_csv("../data/SotoEtAl_ADs.csv")
Soto["Reference"] = "PMID: " + Soto["Reference (PMID)"] + ", Soto"
Soto = Soto[["GeneName", "Start", "End", "uniprotID", "Reference", "Sequence"]]
Soto

,GeneName,Start,End,uniprotID,Reference,Sequence
0,AHCTF1,1445,1698,Q8WYP5,"PMID: 11952839, Soto",IRANDNKSMADVLGDGGNSSLTISEGPIVSERRLNQEVALNLKEDH...
1,AHR,545,848,P35869,"PMID: 8794892, Soto",NLGIDFEDIRHMQNEKFFRNDFSGEVDFRDIDLTDEILTYVQDSLS...
2,AIRE,292,545,O43918,"PMID: 11274163, Soto",HQKNEDECAVCRDGGELICCDGCPRAFHLACLSPPLREIPSGTWRC...
3,AKAP8L,1,384,Q9ULX6,"PMID: 11402034, Soto",MSYTGFVQGSETTLQSTYSDTSAQPTCDYGYGTWNSGTNRGYEGYG...
4,ALX1,237,281,Q15699,"PMID: 12390248, Soto",TSCMLPRDTSSCMTPYSHSPRTDSSYTGFSNHQNQFSHVPLNNFF
...,...,...,...,...,...,...
515,ZNF326,1,124,Q5BKZ1,"PMID: 30401956, Soto",MDFEDDYTHSACRNTYQGFNGMDRDYGPGSYGGMDRDYGHGSYGGQ...
516,ZNF426,99,213,Q9BUY5,"PMID: 17409159, Soto",SRTVQGGVLQGWEMRLETQWSILQQDFLRGQTSIGIQLEGKHNGRE...
517,ZNF641,171,262,Q96N77,"PMID: 16343441, Soto",PDPQDLEERDILRVTYTGDGSEHEGDTPELEAEPPRMLSSVSEDTV...
518,ZXDA,572,698,P98168,"PMID: 17493635, Soto",QDLLAQLEAANSLTPSSELTSQRQNDLSDAEIVSLFSDVPDSTSAA...


In [164]:
Stanford = pd.read_excel("../data/Stanford_Supplementary Table 2 Domains from Tiles.xlsx")
Stanford = Stanford.rename(columns = {"UniProt ID" : "uniprotID", "HGNC symbol" : "GeneName"})
Stanford["Reference"] = "Stanford"
Stanford = Stanford[["GeneName", "Start", "End", "uniprotID", "Reference", "Sequence"]]

# Stanford did zero-indexing, so adjusting to make one-indexed like other lists
# [Start, End] -> [Start, End]
Stanford["Start"] = Stanford["Start"] + 1
Stanford["End"] = Stanford["End"] + 1
Stanford["TF"] = [_ in set(lambert_TFs["uniprotID"]) for _ in Stanford["uniprotID"]]
Stanford

,GeneName,Start,End,uniprotID,Reference,Sequence,TF
0,ABRAXAS1,122,201,Q6UWZ7,Stanford,LQEHFSNQDLVFLLLTPSIITESCSTHRLEHSLYKPQKGLFHRVPL...,False
1,AHR,532,641,P35869,Stanford,QDSKNSDLYSIMKNLGIDFEDIRHMQNEKFFRNDFSGEVDFRDIDL...,True
2,AHR,642,721,P35869,Stanford,MKHMQVNGMFENWNSNQFVPFNCPQQDPQQYNVFTDLHGISQEFPY...,True
3,AKAP8,2,81,O43823,Stanford,DQGYGGYGAWSAGPANTQGAYGTGVASWQGYENYNYYGAQNTSVTT...,True
4,AKAP8L,2,81,Q9ULX6,Stanford,SYTGFVQGSETTLQSTYSDTSAQPTCDYGYGTWNSGTNRGYEGYGY...,True
...,...,...,...,...,...,...,...
369,ZNF532,2,81,Q9HCE3,Stanford,TMGDMKTPDFDDLLAAFDIPDMVDPKAAIESGHDDHESHMKQNAHG...,True
370,ZNF592,2,81,Q92610,Stanford,GDMKTPDFDDLLAAFDIPDPTSLDAKEAIQTPSEENESPLKPPGIC...,True
371,ZNF644,902,981,Q9H582,Stanford,PGENATLSYDQNDGFYFEYYEDTGSNNFLHEIHDPQHLETADASLS...,True
372,ZNF687,2,81,Q8N1G0,Stanford,GDMKTPDFDDLLAAFDIPDIDANEAIHSGPEENEGPGGPGKPEPGV...,True


In [165]:
Stanford_TFs = Stanford[Stanford["uniprotID"].isin(lambert_TFs["uniprotID"])]
Stanford_TFs

,GeneName,Start,End,uniprotID,Reference,Sequence,TF
1,AHR,532,641,P35869,Stanford,QDSKNSDLYSIMKNLGIDFEDIRHMQNEKFFRNDFSGEVDFRDIDL...,True
2,AHR,642,721,P35869,Stanford,MKHMQVNGMFENWNSNQFVPFNCPQQDPQQYNVFTDLHGISQEFPY...,True
3,AKAP8,2,81,O43823,Stanford,DQGYGGYGAWSAGPANTQGAYGTGVASWQGYENYNYYGAQNTSVTT...,True
4,AKAP8L,2,81,Q9ULX6,Stanford,SYTGFVQGSETTLQSTYSDTSAQPTCDYGYGTWNSGTNRGYEGYGY...,True
9,ARGFX,142,251,A6NJG6,Stanford,AKQRNQILPSKKNVPTSPRTSPSPYAFSPVISDFYSSLPSQPLDPS...,True
...,...,...,...,...,...,...,...
369,ZNF532,2,81,Q9HCE3,Stanford,TMGDMKTPDFDDLLAAFDIPDMVDPKAAIESGHDDHESHMKQNAHG...,True
370,ZNF592,2,81,Q92610,Stanford,GDMKTPDFDDLLAAFDIPDPTSLDAKEAIQTPSEENESPLKPPGIC...,True
371,ZNF644,902,981,Q9H582,Stanford,PGENATLSYDQNDGFYFEYYEDTGSNNFLHEIHDPQHLETADASLS...,True
372,ZNF687,2,81,Q8N1G0,Stanford,GDMKTPDFDDLLAAFDIPDIDANEAIHSGPEENEGPGGPGKPEPGV...,True


In [107]:
activity_data = pd.read_csv('../data/Staller2021/SupplementalDataSet4_ActivityData_PredictedADs_renorm20210708_uniprotIDs_added.csv', index_col = 0)
activity_data = activity_data[activity_data["RegionType"] == "Prediction"]
active = activity_data[activity_data["Activity_mean"] > 221]
active["Reference"] = "Staller Activity Data"
active = active[["GeneName", "Start", "End", "uniprotID", "Reference", "ProteinRegionSeq"]]
active = active.rename(columns = {"ProteinRegionSeq" : "Sequence"})
active["GeneName"] = active["GeneName"].str.extract(r'\|.*\|(.*)_')


# Zero-indexing, so adjusting to make one-indexed like other lists
# [Start, End] -> [Start, End)
active["Start"] = active["Start"] + 1
active["End"] = active["End"]
active

,GeneName,Start,End,uniprotID,Reference,Sequence
0,ZN561,29.0,68.0,Q8N587,Staller Activity Data,VEDYLASGYQDSVTFDDVAVDFTPEEWALLDTTEKYLYRD
5,MGAP,2795.0,2855.0,Q8IWI9,Staller Activity Data,VTSAIEEAALDSSELLTNMEDEDDTDETLTSLLNEIAFLNQQLNDD...
15,FOS,209.0,246.0,P01100,Staller Activity Data,DLGFPEEMSVASLDLTGGLPEVATPESEEAFTLPLLND
16,ELF3,134.0,173.0,P78545,Staller Activity Data,DELSWIIELLEKDGMAFQEALDPGPFDQGSPFAQELLDDG
17,ZN644,852.0,893.0,Q9H582,Staller Activity Data,SYETEDESSWDNVELGDYTTQAIEDETYSDINQEHVNLFPLF
19,FOXO4,462.0,505.0,P98177,Staller Activity Data,SQDRMPQDLDLDMYMENLECDMDNIISDLMDEGEGLDFNFEPDP
21,ETV4,40.0,90.0,P43268,Staller Activity Data,MDPGSLPPLDSEDLFQDLSHFQETWLAEAQVPDSDEQFVPDFHSEN...
23,PTF1A,2.0,75.0,Q7RTS3,Staller Activity Data,DAVLLEHFPGGLDAFPSSYFDEDDFFTDQSSRDPLEDGDELLADEQ...
25,MYC,12.0,50.0,P01106,Staller Activity Data,YDLDYDSVQPYFYCDEEENFYQQQQQSELQPPAPSEDIW
28,NFKB1,745.0,788.0,P19838,Staller Activity Data,PLVENFEPLYDLDDSWENAGEDEGVVPGTTPLDMATSWQVFDIL


In [108]:
known_ADs = pd.concat([GSL, Soto, Stanford, active])
known_ADs["Start"] = known_ADs["Start"].astype(int)
known_ADs["End"] = known_ADs["End"].astype(int)
known_ADs = known_ADs.reset_index()
known_ADs

,index,GeneName,Start,End,uniprotID,Reference,Sequence
0,0,MEIS1,335,390,O00470,"R4TA_regions.txt / activation_regions.txt, GSL",IDQSNRAVSQGTPYNPDGQPMGGFVMDGQQHMGIRAPGPMSGMGMN...
1,1,E2F3,391,465,O00716,"activation_regions.txt, GSL",SVSMGNLSPLASPANLLQQTEDQIPSNLEGPFVNLLPPLLQEDYLL...
2,2,MEIS2,340,477,O14770,"activation_regions.txt, GSL",DQSNRAGFLLDPSVSQGAAYSPEGQPMGSFVLDGQQHMGIRPAGLQ...
3,3,TFEC,1,119,O14948,"activation_regions.txt, GSL",MTLDHQIINPTLKWSQPAVPSGGPLVQHAHTTLDSDAGLTENPLTK...
4,4,TFEC,271,347,O14948,"activation_regions.txt, GSL",PSPELCDQAIAFSDPLSYFTDLSFSAALKEEQRLDGMLLDDTISPF...
...,...,...,...,...,...,...,...
1118,143,ETV1,37,83,P50549,Staller Activity Data,RDLAHDSEELFQDLSQLQETWLAEAQVPDNDEQFVPDYQAESLAFHG
1119,145,E2F4,367,407,Q16254,Staller Activity Data,LLEELMSSEVFAPLLRLSPPPGDHDYIYNLDESEGVCDLFD
1120,146,NCOA2,786,825,Q15596,Staller Activity Data,TEKEEMSFEPGDQPGSELDNLEEILDDLQNSQLPQLFPDT
1121,147,NCOA2,1059,1097,Q15596,Staller Activity Data,SPDDLLCPHPAAESPSDEGALLDQLYLALRNFDGLEEID


In [109]:
all_isoform_seqs = protfasta.read_fasta("../data/uniprot_known_AD_seqs_isoforms.txt")
isoform_df = pd.DataFrame({"id":all_isoform_seqs.keys(),
             "seq":all_isoform_seqs.values()})
isoform_df["uniprotID"] = isoform_df["id"].str.split("|").str[1]
isoform_df

,id,seq,uniprotID
0,tr|A0A024R0Y4|A0A024R0Y4_HUMAN Transcriptional...,MDRLGSFSNDPSDKPPCRGCSSYLMEPYIKCAECGPPPFFLCLQCF...,A0A024R0Y4
1,tr|A0A087WXG3|A0A087WXG3_HUMAN Basic helix-loo...,MSIRPPGEPPSPGGAAMAELKSLSGDAYLALSHGYAAAAAGLAYGA...,A0A087WXG3
2,sp|A6NJG6|ARGFX_HUMAN Arginine-fifty homeobox ...,MRNRMAPENPQPDPFINRNYSNMKVIPPQDPASPSFTLLSKLECSG...,A6NJG6
3,sp|A8MTJ6|FOXI3_HUMAN Forkhead box protein I3 ...,MALYCGDNFGVYSQPGLPPPAATAAAPGAPPAARAPYGLADYAAPP...,A8MTJ6
4,sp|A8MW92|P20L1_HUMAN PHD finger protein 20-li...,MSKKPPNRPGITFEIGARLEALDYLQKWYPSRIEKIDYEEGKMLVH...,A8MW92
...,...,...,...
1605,sp|Q9Y6Q9-5|NCOA3_HUMAN Isoform 5 of Nuclear r...,MSGLGENLDPLASDSRKRKLPCDTPGQGLTCSGEKRRREQESKYIE...,Q9Y6Q9-5
1606,sp|Q9Y6Y1|CMTA1_HUMAN Calmodulin-binding trans...,MWRAEGKWLPKTSRKSVSQSVFCGTSTYCVLNTVPPIEDDHGNSNS...,Q9Y6Y1
1607,sp|Q9Y6Y1-2|CMTA1_HUMAN Isoform 2 of Calmoduli...,MWRAEGKWLPKTSRKSVSQSVFCGTSTYCVLNTVPPIEDDHGNSNS...,Q9Y6Y1-2
1608,sp|Q9Y6Y1-3|CMTA1_HUMAN Isoform 3 of Calmoduli...,MWRAEGKWLPKTSRKSVSQSVFCGTSTYCVLNTVPPIEDDHGNSNS...,Q9Y6Y1-3


In [243]:
Stanford_TFs_output = AD_comparison_tools.return_uniprotID_isoform_mappings(Stanford_TFs, isoform_df)
Stanford_TFs_output

no matching isoforms for:
A6NJG6
Tested:
2    A6NJG6
Name: uniprotID, dtype: object
192
315
expected:
A6NJG6
NSTFTESSTSDFQMQDTQWERLVASVPALYSDAYDIFQIIELYNLPDENEISSSSFHCLYQYLSPTKYQVGGQGSSLSIFAGPAVGLSPAQTWPNMTSQAFEAYSLTDSLEFQKTSNMVDTSNMVDLGFL
observed:
['NSTFTESSTSDFQMQDTQWERLVASVPALYSDAYDIFQIIELYNLPDENEISSSSFHCLYQYLSPTKYQVGGQGSSLSIFAGPAVGLSPAQTWPNMTSQAFEAYSLTDSLEFQKTSNMVDLGFL']

no matching isoforms for:
O00327
Tested:
28      O00327
29    O00327-1
30    O00327-3
31    O00327-4
32    O00327-5
33    O00327-6
34    O00327-7
35    O00327-8
36    O00327-9
Name: uniprotID, dtype: object
542
625
expected:
O00327
GGTPDIPSSGLLSGQAQENPGYPYSDSSSILGENPHIGIDMIDNDQGSSSPSNDEAAMAVIMSLLEADAGLGGPVDFSDLVDFSDLPWPL
observed:
['NGGTPDIPSSGLLSGQAQENPGYPYSDSSSILGENPHIGIDMIDNDQGSSSPSNDEAAMAVIMSLLEADAGLGGPVDFSDLPWP', '', '', '', '', '', '', 'GGTPDIPSSGLLSGQAQENPGYPYSDSSSILGENPHIGIDMIDNDQGSSSPSNDEAAMAVIMSLLEADAGLGGPVDFSDLPWPL', '']

no matching isoforms for:
Q8WYA1
Tested:
1270      Q8WYA1
1271    Q8WYA1-2
1272  

no matching isoforms for:
O14948
Tested:
71      O14948
72    O14948-2
73    O14948-3
74    O14948-4
Name: uniprotID, dtype: object
242
347
expected:
O14948
DLGAHVTKQQSHPEQNSVDYCQQLTVSQGPSPELCDQAIAFSDPLSYFTDLSFSAALKEEQRLDGMLLDDTISPFGTDPLLSATSPAVSKESSRRSSFSSSFSSDDGDEL
observed:
['DLGAHVTKQQSHPEQNSVDYCQQLTVSQGPSPELCDQAIAFSDPLSYFTDLSFSAALKEEQRLDGMLLDDTISPFGTDPLLSATSPAVSKESSRRSSFSSDDGDEL', '', '', '']

no matching isoforms for:
P86452
Tested:
750    P86452
Name: uniprotID, dtype: object
731
810
expected:
P86452
TQSSSDSSPPEVSGPSSSQMSMKTRRLSSASTGKPPLSVEDDFEKLIWEISGGKLEAEIDLDPGKDEDDLLLELSEMIDS
observed:
['QVRGITQALNLVDSLSLKLETDTLLSAMLKSKPCILATLLDPCFKNSLEDFFPQGADLETYKQFLAEEVCNYMESSPEIC']



,GeneName,Start,End,uniprotID,Reference,Sequence,TF,matching_isoforms
1,AHR,532,641,P35869,Stanford,QDSKNSDLYSIMKNLGIDFEDIRHMQNEKFFRNDFSGEVDFRDIDL...,True,[P35869]
2,AHR,642,721,P35869,Stanford,MKHMQVNGMFENWNSNQFVPFNCPQQDPQQYNVFTDLHGISQEFPY...,True,[P35869]
3,AKAP8,2,81,O43823,Stanford,DQGYGGYGAWSAGPANTQGAYGTGVASWQGYENYNYYGAQNTSVTT...,True,[O43823]
4,AKAP8L,2,81,Q9ULX6,Stanford,SYTGFVQGSETTLQSTYSDTSAQPTCDYGYGTWNSGTNRGYEGYGY...,True,[Q9ULX6]
9,ARGFX,142,251,A6NJG6,Stanford,AKQRNQILPSKKNVPTSPRTSPSPYAFSPVISDFYSSLPSQPLDPS...,True,[A6NJG6]
...,...,...,...,...,...,...,...,...
369,ZNF532,2,81,Q9HCE3,Stanford,TMGDMKTPDFDDLLAAFDIPDMVDPKAAIESGHDDHESHMKQNAHG...,True,[Q9HCE3]
370,ZNF592,2,81,Q92610,Stanford,GDMKTPDFDDLLAAFDIPDPTSLDAKEAIQTPSEENESPLKPPGIC...,True,[Q92610]
371,ZNF644,902,981,Q9H582,Stanford,PGENATLSYDQNDGFYFEYYEDTGSNNFLHEIHDPQHLETADASLS...,True,"[Q9H582, Q9H582-2]"
372,ZNF687,2,81,Q8N1G0,Stanford,GDMKTPDFDDLLAAFDIPDIDANEAIHSGPEENEGPGGPGKPEPGV...,True,"[Q8N1G0, Q8N1G0-2]"


In [146]:
"Q6UWZ7" in lambert_TFs["uniprotID"]

False

In [166]:
Stanford_output = AD_comparison_tools.return_uniprotID_isoform_mappings(Stanford, isoform_df)
Stanford_output

no matching isoforms for:
Q6UWZ7
Tested:
1157      Q6UWZ7
1158    Q6UWZ7-2
Name: uniprotID, dtype: object
122
201
expected:
Q6UWZ7
LQEHFSNQDLVFLLLTPSIITESCSTHRLEHSLYKPQKGLFHRVPLVVANLGMSEQLGYKTVSGSCMSTGFSRAVQTHSS
observed:
['QEHFSNQDLVFLLLTPSIITESCSTHRLEHSLYKPQKGLFHRVPLVVANLGMSEQLGYKTVSGSCMSTGFSRAVQTHSSK', 'KKVEDSEQAVDKLVKDVNRLKREIEKRRGAQIQAAREKNIQKDPQENIFLCQALRTFFPNSEFLHSCVMSLKNRHVSKSS']

no matching isoforms for:
O00213
Tested:
17      O00213
18    O00213-2
19    O00213-3
20    O00213-4
21    O00213-5
22    O00213-6
Name: uniprotID, dtype: object
222
331
expected:
O00213
DSSWATLSQGSPSYGSPEDTDSFWNPNAFETDSDLPAGWMRVQDTSGTYYWHIPTGTTQWEPPGRASPSQGSSPQEESQLTWTGFAHGEGFEDGEFWKDEPSDEAPMELG
observed:
['SSWATLSQGSPSYGSPEDTDSFWNPNAFETDSDLPAGWMRVQDTSGTYYWHIPTGTTQWEPPGRASPSQGSSPQEESQLTWTGFAHGEGFEDGEFWKDEPSDEAPMELGL', 'SSWATLSQGSPSYGSPEDTDSFWNPNAFETDSDLPAGWMRVQDTSGTYYWHIPTGTTQWEPPGRASPSQGSSPQEESQLTWTGFAHGEGFEDGEFWKDEPSDEAPMELGL', 'LHAQPIISIRVWGVGRDSGRDFAYVARDKLTQMLKCHVFRCEAPAKNIATSLHEICSKIMAERRNARCLV

no matching isoforms for:
Q99741
Tested:
1378    Q99741
Name: uniprotID, dtype: object
172
251
expected:
Q99741
RLPAREREMDVIRNFLREHICGKKAGSLYLSGAPGTGKTACLSRILQDLKKELKGFKTIMLNCMSLRTAQAVFPAIAQEI
observed:
['LPAREREMDVIRNFLREHICGKKAGSLYLSGAPGTGKTACLSRILQDLKKELKGFKTIMLNCMSLRTAQAVFPAIAQEIC']

no matching isoforms for:
Q9BXL8
Tested:
1403    Q9BXL8
Name: uniprotID, dtype: object
132
231
expected:
Q9BXL8
PVTSAQAPRHLQSSAWEMDGPRENRGSFHKSLDQIFETLETKNPSCMEELFSDVDSPYYDLDTVLTGMMGGARPGPCEGLEGLAPATPGPSSSCKSDLGE
observed:
['VTSAQAPRHLQSSAWEMDGPRENRGSFHKSLDQIFETLETKNPSCMEELFSDVDSPYYDLDTVLTGMMGGARPGPCEGLEGLAPATPGPSSSCKSDLGEL']

no matching isoforms for:
Q9BXL8
Tested:
1403    Q9BXL8
Name: uniprotID, dtype: object
163
242
expected:
Q9BXL8
LDQIFETLETKNPSCMEELFSDVDSPYYDLDTVLTGMMGGARPGPCEGLEGLAPATPGPSSSCKSDLGELDHVVEILVET
observed:
['']

no matching isoforms for:
Q00526
Tested:
758    Q00526
Name: uniprotID, dtype: object
142
221
expected:
Q00526
IKLADFGLARAFGVPLRTYTHEVVTLWYRAPEILLGSKFYTTAVDIWSIGCIFAEMVTRKAL

no matching isoforms for:
O95677
Tested:
203      O95677
204    O95677-2
205    O95677-3
206    O95677-4
207    O95677-5
Name: uniprotID, dtype: object
212
311
expected:
O95677
TQSPLQSGCLSYSPGFSTPQPGQTPYSYQMPGSSFAPSSTIYANNSVSNSTNFSGSQQDYPSYTAFGQNQYAQYYSASTYGAYMTSNNTADGTPSSTSTY
observed:
['QSPLQSGCLSYSPGFSTPQPGQTPYSYQMPGSSFAPSSTIYANNSVSNSTNFSGSQQDYPSYTAFGQNQYAQYYSASTYGAYMTSNNTADGTPSSTSTYQ', 'PYSYQMPGSSFAPSSTIYANNSVSNSTNFSGSQQDYPSYTAFGQNQYAQYYSASTYGAYMTSNNTADGTPSSTSTYQLQESLPGLTNQPGEFDTMQSPST', 'QSPLQSGCLSYSPGFSTPQPGQTPYSYQMPGSSFAPSSTIYANNSVSNSTNFSGSQQDYPSYTAFGQNQYAQYYSASTYGAYMTSNNTADGTPSSTSTYQ', 'QSPLQSGCLSYSPGFSTPQPGQTPYSYQMPGSSFAPSSTIYANNSVSNSTNFSGSQQDYPSYTAFGQNQYAQYYSASTYGAYMTSNNTADGTPSSTSTYQ', 'SQQDYPSYTAFGQNQYAQYYSASTYGAYMTSNNTADGTPSSTSTYQLQESLPGLTNQPGEFDTMQSPSTPIKDLDERTCRSSGSKSRGRGRKNNPSPPPD']

no matching isoforms for:
Q12778
Tested:
902    Q12778
Name: uniprotID, dtype: object
552
655
expected:
Q12778
MNRLTQVKTPVQVPLPHPMQMSALGGYSSVSSCNGYGRMGLLHQEKLPSDLDGMFIERLDCDMESIIRNDLMDGDTLDF

no matching isoforms for:
O14686
Tested:
52      O14686
53    O14686-3
Name: uniprotID, dtype: object
2982
3071
expected:
O14686
LEAGKLPCEDPELDDDFDAHKALEDDEELAHLGLGVDVAKGDDELGTLENLETNDPHLDDLLNGDEFDLLAYTDPELDTGDKKDIFNEHL
observed:
['EAGKLPCEDPELDDDFDAHKALEDDEELAHLGLGVDVAKGDDELGTLENLETNDPHLDDLLNGDEFDLLAYTDPELDTGDKKDIFNEHLR', 'LALEAGKLPCEDPELDDDFDAHKALEDDEELAHLGLGVDVAKGDDELGTLENLETNDPHLDDLLNGDEFDLLAYTDPELDTGDKKDIFNE']

no matching isoforms for:
O14686
Tested:
52      O14686
53    O14686-3
Name: uniprotID, dtype: object
3032
3111
expected:
O14686
LETNDPHLDDLLNGDEFDLLAYTDPELDTGDKKDIFNEHLRLVESANEKAEREALLRGVEPGPLGPEERPPPAADASEPR
observed:
['ETNDPHLDDLLNGDEFDLLAYTDPELDTGDKKDIFNEHLRLVESANEKAEREALLRGVEPGPLGPEERPPPAADASEPRL', 'ENLETNDPHLDDLLNGDEFDLLAYTDPELDTGDKKDIFNEHLRLVESANEKAEREALLRGVEPGPLGPEERPPPAADASE']

no matching isoforms for:
Q9Y4W2
Tested:
1586      Q9Y4W2
1587    Q9Y4W2-2
1588    Q9Y4W2-3
1589    Q9Y4W2-4
Name: uniprotID, dtype: object
642
721
expected:
Q9Y4W2
SSEDVRWDTFPLGRMPGQTEDPAEL

no matching isoforms for:
Q9UBK2
Tested:
1505      Q9UBK2
1506    Q9UBK2-2
1507    Q9UBK2-3
1508    Q9UBK2-4
1509    Q9UBK2-5
1510    Q9UBK2-6
1511    Q9UBK2-7
1512    Q9UBK2-8
1513    Q9UBK2-9
Name: uniprotID, dtype: object
2
131
expected:
Q9UBK2
MAWDMCNQDSESVWSDIECAALVGEDQPLCPDLPELDLSELDVNDLDTDSFLGGLKWCSDQSEIISNQYNNEPSNIFEKIDEENEANLLAVLTETLDSLPVDEDGLPSFDALTDGDVTTDNEASPSSMPD
observed:
['AWDMCNQDSESVWSDIECAALVGEDQPLCPDLPELDLSELDVNDLDTDSFLGGLKWCSDQSEIISNQYNNEPSNIFEKIDEENEANLLAVLTETLDSLPVDEDGLPSFDALTDGDVTTDNEASPSSMPDG', 'AWDMCNQDSESVWSDIECAALVGEDQPLCPDLPELDLSELDVNDLDTDSFLGGLKWCSDQSEIISNQYNNEPSNIFEKIDEENEANLLAVLTETLDSLPVDEDGLPSFDALTDGDVTTDNEASPSSMPDG', 'DETSPRLEEDWKKVLQREAGWQCAALVGEDQPLCPDLPELDLSELDVNDLDTDSFLGGLKWCSDQSEIISNQYNNEPSNIFEKIDEENEANLLAVLTETLDSLPVDEDGLPSFDALTDGDVTTDNEASPS', 'DEGYFCAALVGEDQPLCPDLPELDLSELDVNDLDTDSFLGGLKWCSDQSEIISNQYNNEPSNIFEKIDEENEANLLAVLTETLDSLPVDEDGLPSFDALTDGDVTTDNEASPSSMPDGTPPPQEAEEPSL', 'DEGYFCAALVGEDQPLCPDLPELDLSELDVNDLDTDSFLGGLKWCSDQSEIISNQYNNEPSNIFEKIDEENEA

no matching isoforms for:
O14744
Tested:
54      O14744
55    O14744-2
56    O14744-3
57    O14744-4
58    O14744-5
Name: uniprotID, dtype: object
382
471
expected:
O14744
QADRRIKLYAVEKNPNAVVTLENWQFEEWGSQVTVVSSDMREWVAPEKADIIVSELLGSFADNELSPECLDGAQHFLKDDGVSIPGEYTS
observed:
['ADRRIKLYAVEKNPNAVVTLENWQFEEWGSQVTVVSSDMREWVAPEKADIIVSELLGSFADNELSPECLDGAQHFLKDDGVSIPGEYTSF', 'VTLENWQFEEWGSQVTVVSSDMREWVAPEKADIIVSELLGSFADNELSPECLDGAQHFLKDDGVSIPGEYTSFLAPISSSKLYNEVRACR', 'NELSPECLDGAQHFLKDDGVSIPGEYTSFLAPISSSKLYNEVRACREKDRDPEAQFEMPYVVRLHNFHQLSAPQPCFTFSHPNRDPMIDN', '', 'PEKADIIVSELLGSFADNELSPECLDGAQHFLKDDGVSIPGEYTSFLAPISSSKLYNEVRACREKDRDPEAQFEMPYVVRLHNFHQLSAP']

no matching isoforms for:
Q86UA1
Tested:
1180      Q86UA1
1181    Q86UA1-2
Name: uniprotID, dtype: object
392
471
expected:
Q86UA1
IKYAKYMENHSIEGVRHVFSRACTIHLPKKPMVHMLWAAFEEQQGNINEARNILKTFEECVLGLAMVRLRRVSLERRHGN
observed:
['KYAKYMENHSIEGVRHVFSRACTIHLPKKPMVHMLWAAFEEQQGNINEARNILKTFEECVLGLAMVRLRRVSLERRHGNL', '']

no matching isoforms for:
Q9Y4B4


no matching isoforms for:
O14948
Tested:
71      O14948
72    O14948-2
73    O14948-3
74    O14948-4
Name: uniprotID, dtype: object
242
347
expected:
O14948
DLGAHVTKQQSHPEQNSVDYCQQLTVSQGPSPELCDQAIAFSDPLSYFTDLSFSAALKEEQRLDGMLLDDTISPFGTDPLLSATSPAVSKESSRRSSFSSSFSSDDGDEL
observed:
['DLGAHVTKQQSHPEQNSVDYCQQLTVSQGPSPELCDQAIAFSDPLSYFTDLSFSAALKEEQRLDGMLLDDTISPFGTDPLLSATSPAVSKESSRRSSFSSDDGDEL', '', '', '']

no matching isoforms for:
Q04724
Tested:
862    Q04724
Name: uniprotID, dtype: object
602
681
expected:
Q04724
WDLHNQTLVRQFQGHTDGASCIDISNDGTKLWTGGLDNTVRSWDLREGRQLQQHDFTSQIFSLGYCPTGEWLAVGMESSN
observed:
['DLHNQTLVRQFQGHTDGASCIDISNDGTKLWTGGLDNTVRSWDLREGRQLQQHDFTSQIFSLGYCPTGEWLAVGMESSNV']

no matching isoforms for:
Q04726
Tested:
863      Q04726
864    Q04726-2
865    Q04726-3
866    Q04726-4
867    Q04726-5
868    Q04726-6
869    Q04726-7
Name: uniprotID, dtype: object
592
681
expected:
Q04726
CFSCCSDGNIAVWDLHNQTLVRQFQGHTDGASCIDISHDGTKLWTGGLDNTVRSWDLREGRQLQQHDFTSQIFSLGYCPTGEWLAVGMES
observed:


,GeneName,Start,End,uniprotID,Reference,Sequence,TF,matching_isoforms
0,ABRAXAS1,122,201,Q6UWZ7,Stanford,LQEHFSNQDLVFLLLTPSIITESCSTHRLEHSLYKPQKGLFHRVPL...,False,[]
1,AHR,532,641,P35869,Stanford,QDSKNSDLYSIMKNLGIDFEDIRHMQNEKFFRNDFSGEVDFRDIDL...,True,[P35869]
2,AHR,642,721,P35869,Stanford,MKHMQVNGMFENWNSNQFVPFNCPQQDPQQYNVFTDLHGISQEFPY...,True,[P35869]
3,AKAP8,2,81,O43823,Stanford,DQGYGGYGAWSAGPANTQGAYGTGVASWQGYENYNYYGAQNTSVTT...,True,[O43823]
4,AKAP8L,2,81,Q9ULX6,Stanford,SYTGFVQGSETTLQSTYSDTSAQPTCDYGYGTWNSGTNRGYEGYGY...,True,[Q9ULX6]
...,...,...,...,...,...,...,...,...
369,ZNF532,2,81,Q9HCE3,Stanford,TMGDMKTPDFDDLLAAFDIPDMVDPKAAIESGHDDHESHMKQNAHG...,True,[Q9HCE3]
370,ZNF592,2,81,Q92610,Stanford,GDMKTPDFDDLLAAFDIPDPTSLDAKEAIQTPSEENESPLKPPGIC...,True,[Q92610]
371,ZNF644,902,981,Q9H582,Stanford,PGENATLSYDQNDGFYFEYYEDTGSNNFLHEIHDPQHLETADASLS...,True,"[Q9H582, Q9H582-2]"
372,ZNF687,2,81,Q8N1G0,Stanford,GDMKTPDFDDLLAAFDIPDIDANEAIHSGPEENEGPGGPGKPEPGV...,True,"[Q8N1G0, Q8N1G0-2]"


In [184]:
len("GSLPAPAPLWPQSPDASDFLPDTQLFPHFTELLLPLDPLEGSSVSTMTSQYQEGDDSMGKKHSGSQPQEEGGSVNENHSGHSGPRLLLDL")

90

In [185]:
508 - 422

86

In [167]:
Stanford_output["num_matching_isoforms"] = [len(_) for _ in Stanford_output["matching_isoforms"]]

In [168]:
Stanford_no_matches = Stanford_output[Stanford_output["num_matching_isoforms"] == 0]
Stanford_no_matches

,GeneName,Start,End,uniprotID,Reference,Sequence,TF,matching_isoforms,num_matching_isoforms
0,ABRAXAS1,122,201,Q6UWZ7,Stanford,LQEHFSNQDLVFLLLTPSIITESCSTHRLEHSLYKPQKGLFHRVPL...,False,[],0
5,APBB1,222,331,O00213,Stanford,DSSWATLSQGSPSYGSPEDTDSFWNPNAFETDSDLPAGWMRVQDTS...,False,[],0
6,APBB3,12,91,O95704,Stanford,ILVNCDDDLWGDHSLEVEAGLPPGWRKIHDAAGTYYWHVPSGSTQW...,False,[],0
7,APBB3,32,111,O95704,Stanford,LPPGWRKIHDAAGTYYWHVPSGSTQWQRPTWELGDAEDPGTGTEGI...,False,[],0
8,APOBEC4,22,101,Q8WW27,Stanford,WLSFSLDCSNCPYHIRTGEEARVSLTEFCQIFGFPYGTTFPQTKHL...,False,[],0
...,...,...,...,...,...,...,...,...,...
353,WDR77,62,141,Q9BQA1,Stanford,FKDPCAAPNEGFCSAGVQTEAGVADLTWVGERGILVASDSGAVELW...,False,[],0
354,WDR77,264,343,Q9BQA1,Stanford,FSPHSVPFLASLSEDCSLAVLDSSLSELFRSQAHRDFVRDATWSPL...,False,[],0
357,ZBED6,731,810,P86452,Stanford,TQSSSDSSPPEVSGPSSSQMSMKTRRLSSASTGKPPLSVEDDFEKL...,True,[],0
361,ZMYM3,912,1001,Q14202,Stanford,KIVETIEELKVKIPSNPLEADILAMAEMIAEAEELDKASSDLCDLV...,False,[],0


In [179]:
AD_comparison_tools.return_uniprotID_isoform_mappings(Stanford_no_matches_CR_toShare, isoform_df)

no matching isoforms for:
A0A669KBM4
Tested:
Series([], Name: uniprotID, dtype: object)
861
950
expected:
A0A669KBM4
KEFEPSVSQTNESYFPFDDELTQDSIVEELVLMEQQMSMNNSHSYGNCLGMTLQSQSVTPGAPMSSHTSSTHFYHPIHSNGTPIHTPTPT
observed:
[]

no matching isoforms for:
A0A669KBM4
Tested:
Series([], Name: uniprotID, dtype: object)
1371
1459
expected:
A0A669KBM4
DLTNTASDFSSDIRLSSELSGSINDLNTLDPNLLFDPGRQQGQDDEATLEELKNDPLFQQICSESMNSMTSSGFEWIESKKDHPTVEMLG
observed:
[]

no matching isoforms for:
D2CFI5
Tested:
10    D2CFI5
Name: uniprotID, dtype: object
421
507
expected:
D2CFI5
GSLPAPAPLWPQSPDASDFLPDTQLFPHFTELLLPLDPLEGSSVSTMTSQYQEGDDSMGKKHSGSQPQEEGGSVNENHSGHSGPRLLLDL
observed:
['PGSLPAPAPLWPQSPDASDFLPDTQLFPHFTELLLPLDPLEGSSVSTMTSQYQEGDDSMGKKHSGSQPQEEGGSVNENHSGPRLLLD']



,GeneName,Start,End,uniprotID,Reference,Sequence,TF,matching_isoforms,num_matching_isoforms
0,ABRAXAS1,121,200,Q6UWZ7,Stanford,LQEHFSNQDLVFLLLTPSIITESCSTHRLEHSLYKPQKGLFHRVPL...,False,[Q6UWZ7],0
5,APBB1,221,330,O00213,Stanford,DSSWATLSQGSPSYGSPEDTDSFWNPNAFETDSDLPAGWMRVQDTS...,False,"[O00213, O00213-2]",0
6,APBB3,11,90,O95704,Stanford,ILVNCDDDLWGDHSLEVEAGLPPGWRKIHDAAGTYYWHVPSGSTQW...,False,"[O95704, O95704-2, O95704-3, O95704-4, O95704-...",0
7,APBB3,31,110,O95704,Stanford,LPPGWRKIHDAAGTYYWHVPSGSTQWQRPTWELGDAEDPGTGTEGI...,False,"[O95704, O95704-2, O95704-3, O95704-4, O95704-...",0
8,APOBEC4,21,100,Q8WW27,Stanford,WLSFSLDCSNCPYHIRTGEEARVSLTEFCQIFGFPYGTTFPQTKHL...,False,[Q8WW27],0
...,...,...,...,...,...,...,...,...,...
352,USP22,271,380,Q9UPT9,Stanford,LDVLHRHCKGDDNGKKANNPNHCNCIIDQIFTGGLQSDVTCQVCHG...,False,[Q9UPT9],0
353,WDR77,61,140,Q9BQA1,Stanford,FKDPCAAPNEGFCSAGVQTEAGVADLTWVGERGILVASDSGAVELW...,False,[Q9BQA1],0
354,WDR77,263,342,Q9BQA1,Stanford,FSPHSVPFLASLSEDCSLAVLDSSLSELFRSQAHRDFVRDATWSPL...,False,[Q9BQA1],0
361,ZMYM3,911,1000,Q14202,Stanford,KIVETIEELKVKIPSNPLEADILAMAEMIAEAEELDKASSDLCDLV...,False,[Q14202],0


In [169]:
# The 25 Stanford TFs with no matches are the ones with the insertion gone wrong
sum(Stanford_no_matches["TF"])

25

In [178]:
# The rest of the entries with no matches (that are not TFs) seem to have an indexing issue
Stanford_no_matches_CR_toShare = Stanford_no_matches[~Stanford_no_matches["TF"]]
Stanford_no_matches_CR_toShare["Start"] -= 1
Stanford_no_matches_CR_toShare["End"] -= 1
Stanford_no_matches_CR_toShare

,GeneName,Start,End,uniprotID,Reference,Sequence,TF,matching_isoforms,num_matching_isoforms
0,ABRAXAS1,121,200,Q6UWZ7,Stanford,LQEHFSNQDLVFLLLTPSIITESCSTHRLEHSLYKPQKGLFHRVPL...,False,[],0
5,APBB1,221,330,O00213,Stanford,DSSWATLSQGSPSYGSPEDTDSFWNPNAFETDSDLPAGWMRVQDTS...,False,[],0
6,APBB3,11,90,O95704,Stanford,ILVNCDDDLWGDHSLEVEAGLPPGWRKIHDAAGTYYWHVPSGSTQW...,False,[],0
7,APBB3,31,110,O95704,Stanford,LPPGWRKIHDAAGTYYWHVPSGSTQWQRPTWELGDAEDPGTGTEGI...,False,[],0
8,APOBEC4,21,100,Q8WW27,Stanford,WLSFSLDCSNCPYHIRTGEEARVSLTEFCQIFGFPYGTTFPQTKHL...,False,[],0
...,...,...,...,...,...,...,...,...,...
352,USP22,271,380,Q9UPT9,Stanford,LDVLHRHCKGDDNGKKANNPNHCNCIIDQIFTGGLQSDVTCQVCHG...,False,[],0
353,WDR77,61,140,Q9BQA1,Stanford,FKDPCAAPNEGFCSAGVQTEAGVADLTWVGERGILVASDSGAVELW...,False,[],0
354,WDR77,263,342,Q9BQA1,Stanford,FSPHSVPFLASLSEDCSLAVLDSSLSELFRSQAHRDFVRDATWSPL...,False,[],0
361,ZMYM3,911,1000,Q14202,Stanford,KIVETIEELKVKIPSNPLEADILAMAEMIAEAEELDKASSDLCDLV...,False,[],0


In [181]:
# Dropping because no match since uniprotID is now obsolete
Stanford_no_matches_CR_toShare[Stanford_no_matches_CR_toShare["uniprotID"].str.contains("A0A669KBM4")]

,GeneName,Start,End,uniprotID,Reference,Sequence,TF,matching_isoforms,num_matching_isoforms
284,RFX7,861,950,A0A669KBM4,Stanford,KEFEPSVSQTNESYFPFDDELTQDSIVEELVLMEQQMSMNNSHSYG...,False,[],0
285,RFX7,1371,1459,A0A669KBM4,Stanford,DLTNTASDFSSDIRLSSELSGSINDLNTLDPNLLFDPGRQQGQDDE...,False,[],0


In [186]:
# Dropping because issue seems to be because of the C-termnus problem
Stanford_no_matches_CR_toShare[Stanford_no_matches_CR_toShare["uniprotID"].str.contains("D2CFI5")]

,GeneName,Start,End,uniprotID,Reference,Sequence,TF,matching_isoforms,num_matching_isoforms
345,TPRX1,421,507,D2CFI5,Stanford,GSLPAPAPLWPQSPDASDFLPDTQLFPHFTELLLPLDPLEGSSVST...,False,[],0


In [191]:
Stanford_no_matches_CR_toShare = Stanford_no_matches_CR_toShare.drop(labels = [284, 285, 345])
Stanford_no_matches_CR_toShare[["GeneName","Start","End","uniprotID","Sequence"]].to_csv("../output/120ADs_seq_mismatch.csv")
Stanford_no_matches_CR_toShare

,GeneName,Start,End,uniprotID,Reference,Sequence,TF,matching_isoforms,num_matching_isoforms
0,ABRAXAS1,121,200,Q6UWZ7,Stanford,LQEHFSNQDLVFLLLTPSIITESCSTHRLEHSLYKPQKGLFHRVPL...,False,[Q6UWZ7],0
5,APBB1,221,330,O00213,Stanford,DSSWATLSQGSPSYGSPEDTDSFWNPNAFETDSDLPAGWMRVQDTS...,False,"[O00213, O00213-2]",0
6,APBB3,11,90,O95704,Stanford,ILVNCDDDLWGDHSLEVEAGLPPGWRKIHDAAGTYYWHVPSGSTQW...,False,"[O95704, O95704-2, O95704-3, O95704-4, O95704-...",0
7,APBB3,31,110,O95704,Stanford,LPPGWRKIHDAAGTYYWHVPSGSTQWQRPTWELGDAEDPGTGTEGI...,False,"[O95704, O95704-2, O95704-3, O95704-4, O95704-...",0
8,APOBEC4,21,100,Q8WW27,Stanford,WLSFSLDCSNCPYHIRTGEEARVSLTEFCQIFGFPYGTTFPQTKHL...,False,[Q8WW27],0
...,...,...,...,...,...,...,...,...,...
352,USP22,271,380,Q9UPT9,Stanford,LDVLHRHCKGDDNGKKANNPNHCNCIIDQIFTGGLQSDVTCQVCHG...,False,[Q9UPT9],0
353,WDR77,61,140,Q9BQA1,Stanford,FKDPCAAPNEGFCSAGVQTEAGVADLTWVGERGILVASDSGAVELW...,False,[Q9BQA1],0
354,WDR77,263,342,Q9BQA1,Stanford,FSPHSVPFLASLSEDCSLAVLDSSLSELFRSQAHRDFVRDATWSPL...,False,[Q9BQA1],0
361,ZMYM3,911,1000,Q14202,Stanford,KIVETIEELKVKIPSNPLEADILAMAEMIAEAEELDKASSDLCDLV...,False,[Q14202],0


In [170]:
Stanford_matches = Stanford_output[Stanford_output["num_matching_isoforms"] > 0]
Stanford_matches

,GeneName,Start,End,uniprotID,Reference,Sequence,TF,matching_isoforms,num_matching_isoforms
1,AHR,532,641,P35869,Stanford,QDSKNSDLYSIMKNLGIDFEDIRHMQNEKFFRNDFSGEVDFRDIDL...,True,[P35869],1
2,AHR,642,721,P35869,Stanford,MKHMQVNGMFENWNSNQFVPFNCPQQDPQQYNVFTDLHGISQEFPY...,True,[P35869],1
3,AKAP8,2,81,O43823,Stanford,DQGYGGYGAWSAGPANTQGAYGTGVASWQGYENYNYYGAQNTSVTT...,True,[O43823],1
4,AKAP8L,2,81,Q9ULX6,Stanford,SYTGFVQGSETTLQSTYSDTSAQPTCDYGYGTWNSGTNRGYEGYGY...,True,[Q9ULX6],1
9,ARGFX,142,251,A6NJG6,Stanford,AKQRNQILPSKKNVPTSPRTSPSPYAFSPVISDFYSSLPSQPLDPS...,True,[A6NJG6],1
...,...,...,...,...,...,...,...,...,...
369,ZNF532,2,81,Q9HCE3,Stanford,TMGDMKTPDFDDLLAAFDIPDMVDPKAAIESGHDDHESHMKQNAHG...,True,[Q9HCE3],1
370,ZNF592,2,81,Q92610,Stanford,GDMKTPDFDDLLAAFDIPDPTSLDAKEAIQTPSEENESPLKPPGIC...,True,[Q92610],1
371,ZNF644,902,981,Q9H582,Stanford,PGENATLSYDQNDGFYFEYYEDTGSNNFLHEIHDPQHLETADASLS...,True,"[Q9H582, Q9H582-2]",2
372,ZNF687,2,81,Q8N1G0,Stanford,GDMKTPDFDDLLAAFDIPDIDANEAIHSGPEENEGPGGPGKPEPGV...,True,"[Q8N1G0, Q8N1G0-2]",2


In [171]:
sum(Stanford_matches["TF"])

217

In [172]:
Stanford_matches[~Stanford_matches["TF"]]

,GeneName,Start,End,uniprotID,Reference,Sequence,TF,matching_isoforms,num_matching_isoforms
27,BHLHE23,162,241,A0A087WXG3,Stanford,LAKNYILMQAQALDEMRRLVAFLNQGQGLAAPVNAAPLTPFGQATV...,False,[A0A087WXG3],1
68,DLX3,2,81,O60479,Stanford,SGSFDRKLSSILTDISSSLSCHAGSKDSPTLPESSVTDLGYYSAPQ...,False,[O60479],1
168,L3MBTL4,472,551,F8W9S8,Stanford,EDIDLDNLFREYSVEQAQQVLHQSVSMSTVSAHPFRDLPLGREQHC...,False,[F8W9S8],1
344,TPRX1,382,481,D2CFI5,Stanford,GPILSPGRMRSPGSLPGLAPILGPGSGPGSGSVPAPIPGPGSLPAP...,False,[D2CFI5],1
358,ZBTB44,502,581,H0YEM9,Stanford,LIVHLRSLNHEASELANYFQSSDFLVPDYLNQEQEETLVQYDLGEH...,False,[H0YEM9],1
365,ZNF469,792,891,H3BS19,Stanford,SHAKTFLLAGDAQAEGKDDPLRTGFLPSLAATPFPLPASDLDMEDD...,False,[H3BS19],1
366,ZNF469,832,911,H3BS19,Stanford,LDMEDDAKLDSLITEALNGMEYQSDNPEIDSSFIDVFADEEPSGPR...,False,[H3BS19],1
367,ZNF469,2892,2971,H3BS19,Stanford,QPSFEEGGDPTLGPARLPTDLSDSSSLCLCHEDPWEDEDPAGLPES...,False,[H3BS19],1
368,ZNF469,3002,3091,H3BS19,Stanford,MPAPADDSSSSLGDVSPEPPSLERERCDGGLPGNTHLLPLRATDFE...,False,[H3BS19],1


In [193]:
# It looks like Stanford sequences might not be accurate - 
# going to move forward by assuming that the uniprotIDs are canonical, and that the coordinates are correct.
Stanford

,GeneName,Start,End,uniprotID,Reference,Sequence,TF,matching_isoforms,num_matching_isoforms
0,ABRAXAS1,122,201,Q6UWZ7,Stanford,LQEHFSNQDLVFLLLTPSIITESCSTHRLEHSLYKPQKGLFHRVPL...,False,[],0
1,AHR,532,641,P35869,Stanford,QDSKNSDLYSIMKNLGIDFEDIRHMQNEKFFRNDFSGEVDFRDIDL...,True,[P35869],1
2,AHR,642,721,P35869,Stanford,MKHMQVNGMFENWNSNQFVPFNCPQQDPQQYNVFTDLHGISQEFPY...,True,[P35869],1
3,AKAP8,2,81,O43823,Stanford,DQGYGGYGAWSAGPANTQGAYGTGVASWQGYENYNYYGAQNTSVTT...,True,[O43823],1
4,AKAP8L,2,81,Q9ULX6,Stanford,SYTGFVQGSETTLQSTYSDTSAQPTCDYGYGTWNSGTNRGYEGYGY...,True,[Q9ULX6],1
...,...,...,...,...,...,...,...,...,...
369,ZNF532,2,81,Q9HCE3,Stanford,TMGDMKTPDFDDLLAAFDIPDMVDPKAAIESGHDDHESHMKQNAHG...,True,[Q9HCE3],1
370,ZNF592,2,81,Q92610,Stanford,GDMKTPDFDDLLAAFDIPDPTSLDAKEAIQTPSEENESPLKPPGIC...,True,[Q92610],1
371,ZNF644,902,981,Q9H582,Stanford,PGENATLSYDQNDGFYFEYYEDTGSNNFLHEIHDPQHLETADASLS...,True,"[Q9H582, Q9H582-2]",2
372,ZNF687,2,81,Q8N1G0,Stanford,GDMKTPDFDDLLAAFDIPDIDANEAIHSGPEENEGPGGPGKPEPGV...,True,"[Q8N1G0, Q8N1G0-2]",2


In [137]:
AD_comparison_tools.return_uniprotID_isoform_mappings(active, isoform_df)
# It looks like the coordinates for Q8IZM8 that would match the sequence are actually starting at 680
# Based on BLAST and alignment

,GeneName,Start,End,uniprotID,Reference,Sequence,matching_isoforms
0,ZN561,29.0,68.0,Q8N587,Staller Activity Data,VEDYLASGYQDSVTFDDVAVDFTPEEWALLDTTEKYLYRD,[Q8N587]
5,MGAP,2795.0,2855.0,Q8IWI9,Staller Activity Data,VTSAIEEAALDSSELLTNMEDEDDTDETLTSLLNEIAFLNQQLNDD...,[Q8IWI9]
15,FOS,209.0,246.0,P01100,Staller Activity Data,DLGFPEEMSVASLDLTGGLPEVATPESEEAFTLPLLND,[P01100]
16,ELF3,134.0,173.0,P78545,Staller Activity Data,DELSWIIELLEKDGMAFQEALDPGPFDQGSPFAQELLDDG,[P78545]
17,ZN644,852.0,893.0,Q9H582,Staller Activity Data,SYETEDESSWDNVELGDYTTQAIEDETYSDINQEHVNLFPLF,"[Q9H582, Q9H582-2]"
19,FOXO4,462.0,505.0,P98177,Staller Activity Data,SQDRMPQDLDLDMYMENLECDMDNIISDLMDEGEGLDFNFEPDP,[P98177]
21,ETV4,40.0,90.0,P43268,Staller Activity Data,MDPGSLPPLDSEDLFQDLSHFQETWLAEAQVPDSDEQFVPDFHSEN...,[P43268]
23,PTF1A,2.0,75.0,Q7RTS3,Staller Activity Data,DAVLLEHFPGGLDAFPSSYFDEDDFFTDQSSRDPLEDGDELLADEQ...,[Q7RTS3]
25,MYC,12.0,50.0,P01106,Staller Activity Data,YDLDYDSVQPYFYCDEEENFYQQQQQSELQPPAPSEDIW,[P01106-1]
28,NFKB1,745.0,788.0,P19838,Staller Activity Data,PLVENFEPLYDLDDSWENAGEDEGVVPGTTPLDMATSWQVFDIL,[P19838]


In [138]:
active[active["uniprotID"].str.contains("Q8IZM8")]

,GeneName,Start,End,uniprotID,Reference,Sequence,matching_isoforms
89,ZN654,680.0,724.0,Q8IZM8,Staller Activity Data,GSPNNSLNNVFKPLTECGDDYEEEEDEEGDYEEDDYDLNQETSVI,[Q8IZM8]


In [139]:
active.at[89, "Start"] = 680
active.at[89, "End"] = 680 + (177-132) - 1

In [244]:
# Trying active again, now it works
active_mappings = AD_comparison_tools.return_uniprotID_isoform_mappings(active, isoform_df)
# Soto and GSL have no issues with mapping
Soto_mappings = AD_comparison_tools.return_uniprotID_isoform_mappings(Soto, isoform_df)
GSL_mappings = AD_comparison_tools.return_uniprotID_isoform_mappings(GSL, isoform_df)

In [198]:
# Updating uniprotID to be the isoform found

for df in [active_mappings, Soto_mappings, GSL_mappings]:
    df["old_uniprotID"] = df["uniprotID"]
    df["uniprotID"] = [_[0] for _ in df["matching_isoforms"]]

In [202]:
def return_merged_row(uniprotID, df):
    # Only look at rows with the same uniprot ID
    same_uniprotID_rows = df[df["uniprotID"] == uniprotID]
    same_uniprotID_rows = same_uniprotID_rows.sort_values(by = "Start")
    
    # Final dataframe columns
    new_starts = []
    new_ends = []
    genes = []
    AD_names = []
    references = []
    
    # Current row's values
    curr_start = -1
    curr_end = -1
    curr_genes = []
    curr_AD_names = []
    curr_references = []
    
    for i in same_uniprotID_rows.index:
        # Merge current row with next row
        if curr_end >= same_uniprotID_rows.loc[i]["Start"]:
            curr_end = max(curr_end, same_uniprotID_rows.loc[i]["End"])
            curr_genes.append(same_uniprotID_rows.loc[i]["GeneName"])
            curr_references.append(same_uniprotID_rows.loc[i]["Reference"])
        
        # Don't merge current row with next row
        else: 
            new_starts.append(curr_start)
            new_ends.append(curr_end)
            genes.append(" / ".join(set([c.strip() for c in curr_genes])))
            
            curr_AD_names = [str(c) for c in curr_AD_names]
            AD_names.append(" / ".join(curr_AD_names))
            
            curr_references = [str(c) for c in curr_references]
            references.append(" / ".join(curr_references))
            
            curr_start = same_uniprotID_rows.loc[i]["Start"]
            curr_end = same_uniprotID_rows.loc[i]["End"]
            
            curr_genes = [same_uniprotID_rows.loc[i]["GeneName"]]
            curr_references = [same_uniprotID_rows.loc[i]["Reference"]]
    
    # Append the last values
    new_starts.append(curr_start)
    
    new_ends.append(curr_end)
    
    genes.append(" / ".join(set([c.strip() for c in curr_genes])))
    
    curr_AD_names = [str(c) for c in curr_AD_names]
    AD_names.append(" / ".join(curr_AD_names))
    
    curr_references = [str(c) for c in curr_references]
    references.append(" / ".join(curr_references))
    
    # Remove the first (because it is just -1 or "")
    new_starts = new_starts[1:]
    new_ends = new_ends[1:]
    genes = genes[1:]
    references = references[1:]

    return pd.DataFrame({"Gene": genes,
                         "Start": new_starts,
                        "End": new_ends,
                        "uniprotID": uniprotID,
                         "Reference": references
                        })

In [214]:
# GeneName, Start, End, uniprotID, Reference, Sequence

mapped_known_ADs = pd.DataFrame()

for df in [active_mappings, Soto_mappings, GSL_mappings, Stanford]:
    mapped_known_ADs = mapped_known_ADs.append(df[["GeneName", "Start", "End", "uniprotID", "Reference", "Sequence"]])
    
mapped_known_ADs = mapped_known_ADs.sort_values(by = "uniprotID")
mapped_known_ADs.reset_index(inplace= True,drop=True)
mapped_known_ADs

,GeneName,Start,End,uniprotID,Reference,Sequence
0,TADA2A,112.0,211.0,A0A024R0Y4,Stanford,EKHYMKHFINNPLFASTLLNLKQAEEAKTADTAIPFHSTDDPPRPT...
1,BHLHE23,162.0,241.0,A0A087WXG3,Stanford,LAKNYILMQAQALDEMRRLVAFLNQGQGLAAPVNAAPLTPFGQATV...
2,RFX7,1372.0,1460.0,A0A669KBM4,Stanford,DLTNTASDFSSDIRLSSELSGSINDLNTLDPNLLFDPGRQQGQDDE...
3,RFX7,862.0,951.0,A0A669KBM4,Stanford,KEFEPSVSQTNESYFPFDDELTQDSIVEELVLMEQQMSMNNSHSYG...
4,ARGFX,142.0,251.0,A6NJG6,Stanford,AKQRNQILPSKKNVPTSPRTSPSPYAFSPVISDFYSSLPSQPLDPS...
...,...,...,...,...,...,...
1118,TAF6L,372.0,481.0,Q9Y6J9,Stanford,KMKAQAAEPNRGGPGGRGCRRLDDLPWDSLLFQESSSGGGAEPSFG...
1119,NCOA3,621.0,1424.0,Q9Y6Q9,"PMID: 9765300, Soto",LLQLLTCSSDDRGHSSLTNSPLDSSCKESSVSVTSPSGVSSSTSGG...
1120,NCOA3,1002.0,1131.0,Q9Y6Q9,Stanford,AASNQLGSWPDGMLSMEQVSHGTQNRPLLRNSLDDLVGPPSNLEGQ...
1121,CAMTA1,702.0,841.0,Q9Y6Y1,Stanford,EVLLKSGELQACSSEHYLQPETNGVIRSAGGVPILPGNVVQGLYPV...


In [215]:
dfs = []
i = 0
for uniprotID in mapped_known_ADs["uniprotID"].unique():
    dfs.append(return_merged_row(uniprotID, mapped_known_ADs))

In [219]:
merged_mapped_known_ADs = pd.concat(dfs)
merged_mapped_known_ADs = merged_mapped_known_ADs.reset_index(drop = True)
merged_mapped_known_ADs

,Gene,Start,End,uniprotID,Reference
0,TADA2A,112.0,211.0,A0A024R0Y4,Stanford
1,BHLHE23,162.0,241.0,A0A087WXG3,Stanford
2,RFX7,862.0,951.0,A0A669KBM4,Stanford
3,RFX7,1372.0,1460.0,A0A669KBM4,Stanford
4,ARGFX,142.0,315.0,A6NJG6,Stanford / Stanford
...,...,...,...,...,...
744,PARP3,232.0,351.0,Q9Y6F1,Stanford
745,TAF6L,372.0,481.0,Q9Y6J9,Stanford
746,NCOA3,621.0,1424.0,Q9Y6Q9,"PMID: 9765300, Soto / Stanford"
747,CAMTA1,422.0,501.0,Q9Y6Y1,Stanford


In [220]:
merged_mapped_known_ADs[merged_mapped_known_ADs["uniprotID"].str.contains("-")]

,Gene,Start,End,uniprotID,Reference
34,MEIS2,340.0,470.0,O14770-4,"PMID: 20553494, Soto"
56,TBX1,409.0,495.0,O43435-3,"PMID: 15703190, Soto"
105,MYC,1.0,144.0,P01106-1,"PMID: 8755740, Soto / Choi 2000 list / PMID: 2..."
128,Oct2,422.0,463.0,P09086-3,"Choi 2000 list, GSL"
144,RARB,1.0,80.0,P10826-2,"PMID: 8389696, Soto"
145,RARB,147.0,448.0,P10826-2,"PMID: 8389696, Soto"
147,THRA,376.0,410.0,P10827-2,"PMID: 8039499, Soto"
154,ERG,125.0,209.0,P11308-1,"PMID: 14603248, Soto"
155,ERG,440.0,486.0,P11308-3,"PMID: 9681824, Soto"
173,c-ETS1,175.0,286.0,P14921-3,"Choi 2000 list, GSL"


In [226]:
merged_mapped_known_ADs.to_csv("../data/known_ADs_isoforms.csv")

In [229]:
# To check work
test = pd.read_csv("../output/known_ADs_isoforms_with_seqs.csv")
test = test.rename(columns = {"ProteinRegionSeq":"Sequence"})

In [231]:
# Only expect error messages from A0A669KBM4
return_uniprotID_isoform_mappings(test, isoform_df)

no matching isoforms for:
A0A669KBM4
Tested:
Series([], Name: uniprotID, dtype: object)
862
951
expected:
A0A669KBM4
KEFEPSVSQTNESYFPFDDELTQDSIVEELVLMEQQMSMNNSHSYGNCLGMTLQSQSVTPGAPMSSHTSSTHFYHPIHSNGTPIHTPTPT
observed:
[]

no matching isoforms for:
A0A669KBM4
Tested:
Series([], Name: uniprotID, dtype: object)
1372
1460
expected:
A0A669KBM4
DLTNTASDFSSDIRLSSELSGSINDLNTLDPNLLFDPGRQQGQDDEATLEELKNDPLFQQICSESMNSMTSSGFEWIESKKDHPTVEMLG
observed:
[]

no matching isoforms for:
Q08050
Tested:
893      Q08050
894    Q08050-2
895    Q08050-3
Name: uniprotID, dtype: object
711
801
expected:
Q08050
GLAANRSLTEGLVLDTMNDSLSKILLDISFPGLDEDPLGPDNINWSQFIPELQ
observed:
['', '', 'PQRLLSSEPLDLISVPFGNSSPSDIDVPKPGSPEPQVSGLAANRSLTEGLVLDTMNDSLSKILLDISFPGLDEDPLGPDNINWSQFIPELQ']

no matching isoforms for:
Q9BQA1
Tested:
1388      Q9BQA1
1389    Q9BQA1-2
Name: uniprotID, dtype: object
264
343
expected:
Q9BQA1
SPHSVPFLASLSEDCSLAVLDSSLSELFRSQAHRDFVRDATWSPLNHSLLTTVGWDHQVVHHVVPTEPLPAPGPASVTE
observed:
['', '']

no matchi

,Unnamed: 0,Gene,Start,End,uniprotID,Reference,Sequence,matching_isoforms
0,0,TADA2A,112,211,A0A024R0Y4,Stanford,KHYMKHFINNPLFASTLLNLKQAEEAKTADTAIPFHSTDDPPRPTF...,[A0A024R0Y4]
1,1,BHLHE23,162,241,A0A087WXG3,Stanford,LAKNYILMQAQALDEMRRLVAFLNQGQGLAAPVNAAPLTPFGQATV...,[A0A087WXG3]
2,2,RFX7,862,951,A0A669KBM4,Stanford,KEFEPSVSQTNESYFPFDDELTQDSIVEELVLMEQQMSMNNSHSYG...,[]
3,3,RFX7,1372,1460,A0A669KBM4,Stanford,DLTNTASDFSSDIRLSSELSGSINDLNTLDPNLLFDPGRQQGQDDE...,[]
4,4,ARGFX,142,315,A6NJG6,Stanford / Stanford,AKQRNQILPSKKNVPTSPRTSPSPYAFSPVISDFYSSLPSQPLDPS...,[A6NJG6]
...,...,...,...,...,...,...,...,...
744,744,PARP3,232,351,Q9Y6F1,Stanford,ALEALEEALKGPTDGGQSLEELSSHFYTVIPHNFGHSQPPPINSPE...,[Q9Y6F1]
745,745,TAF6L,372,481,Q9Y6J9,Stanford,MKAQAAEPNRGGPGGRGCRRLDDLPWDSLLFQESSSGGGAEPSFGS...,[Q9Y6J9]
746,746,NCOA3,621,1424,Q9Y6Q9,"PMID: 9765300, Soto / Stanford",LLQLLTCSSDDRGHSSLTNSPLDSSCKESSVSVTSPSGVSSSTSGG...,[Q9Y6Q9]
747,747,CAMTA1,422,501,Q9Y6Y1,Stanford,LLSPDASQGLVLAVSSDGHKFAFPTTGSSESLSMLPTNVSEELVLS...,"[Q9Y6Y1, Q9Y6Y1-2]"


In [240]:
test[test["uniprotID"] == "Q08050"]
# Q08050 has max length of 763 -> isoform issue, seems like it matches non-canonical isoform

,Unnamed: 0,Gene,Start,End,uniprotID,Reference,Sequence,matching_isoforms
407,407,FOXM1,711,801,Q08050,"PMID: 31134895, Soto / Stanford",GLAANRSLTEGLVLDTMNDSLSKILLDISFPGLDEDPLGPDNINWS...,[]


In [236]:
Stanford[Stanford["uniprotID"] == "Q08050"]
# Q08050 has max length of 763 -> isoform issue, seems like it matches non-canonical isoform

,GeneName,Start,End,uniprotID,Reference,Sequence,TF,matching_isoforms,num_matching_isoforms
108,FOXM1,722,801,Q08050,Stanford,LISVPFGNSSPSDIDVPKPGSPEPQVSGLAANRSLTEGLVLDTMND...,True,[Q08050-3],1


In [239]:
test[test["uniprotID"] == "Q9BQA1"]
# Q9BQA1 has a max length of 342 -> indexing issue

,Unnamed: 0,Gene,Start,End,uniprotID,Reference,Sequence,matching_isoforms
635,635,WDR77,62,141,Q9BQA1,Stanford,KDPCAAPNEGFCSAGVQTEAGVADLTWVGERGILVASDSGAVELWE...,[Q9BQA1]
636,636,WDR77,264,343,Q9BQA1,Stanford,SPHSVPFLASLSEDCSLAVLDSSLSELFRSQAHRDFVRDATWSPLN...,[]


In [242]:
Stanford[Stanford["uniprotID"] == "Q9BXL8"]
# Q9BXL8 has max length of 241 -> indexing issue

,GeneName,Start,End,uniprotID,Reference,Sequence,TF,matching_isoforms,num_matching_isoforms
40,CDCA4,132,231,Q9BXL8,Stanford,PVTSAQAPRHLQSSAWEMDGPRENRGSFHKSLDQIFETLETKNPSC...,False,[],0
41,CDCA4,163,242,Q9BXL8,Stanford,LDQIFETLETKNPSCMEELFSDVDSPYYDLDTVLTGMMGGARPGPC...,False,[],0
